# PA005: High Value Customer Identification

# 0.0 Imports

In [1]:
import re
import numpy   as np
import pandas  as pd
import seaborn as sns

import umap.umap_ as umap

from matplotlib import pyplot as plt


from sklearn import cluster       as c
from sklearn import metrics       as m
from sklearn import ensemble      as en
from sklearn import preprocessing as pp
from sklearn import decomposition as dd
from sklearn import manifold      as mn
from sklearn import mixture       as mx

from plotly import express as px

## 0.2. Load Dataset

In [2]:
# load data
path = '/home/leandro/repos/insiders_clustering/'
df_raw = pd.read_csv(path+'/data/raw/Ecommerce.csv')

df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


# 1.0. Descrição dos dados

In [3]:
df1 = df_raw.copy()

In [4]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


## 1.1. Rename Columns

In [5]:
# Rename Columns
cols_new = ['invoice_no','stock_code','description','quantity','invoice_date','unit_price','customer_id','country']

df1.columns = cols_new

df1.sample()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
522448,580399,82600,N0 SINGING METAL SIGN,2,2-Dec-17,2.1,13471.0,United Kingdom


In [6]:
df_raw.sample()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
287294,562101,22738,RIBBON REEL SNOWY VILLAGE,10,31-Jul-17,1.65,13850.0,United Kingdom


## 1.2. Data Dimensions

In [7]:
print( 'Number of rows: {}'.format ( df1.shape[0] ) )
print( 'Number of cols: {}'.format ( df1.shape[1] ) )


Number of rows: 541909
Number of cols: 8


## 1.3. Data Types

In [8]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [9]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5. Replace NA

In [10]:
df_missing = df1.loc[ df1['customer_id'].isna(), : ]
df_not_missing = df1.loc[~df1['customer_id'].isna(), : ]

In [11]:
# Create Reference
df_backup = pd.DataFrame( df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange( 19000, 19000 +len( df_backup),1)

# Merge original with reference dataframe
df1 = pd.merge( df1, df_backup, on= 'invoice_no', how= 'left' )

# Coalesce
df1[ 'customer_id'] = df1['customer_id_x'].combine_first( df1[ 'customer_id_y' ] )

# Drop extra columns

df1 = df1.drop( columns = ['customer_id_x', 'customer_id_y'], axis = 1)


In [12]:
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.6. Change Dtypes

In [13]:
# Invoice Date

df1['invoice_date'] = pd.to_datetime( df1['invoice_date'], format = '%d-%b-%y')

# Customer Id

df1['customer_id'] = df1['customer_id'].astype(int)

df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850


In [14]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

## 1.7. Descriptive Statistics

In [15]:
num_attributes = df1.select_dtypes( include = [ 'int64', 'float64'] )
cat_attributes = df1.select_dtypes( exclude = [ 'int64', 'float64','datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [16]:
# Central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply( np.mean )).T
ct2 = pd.DataFrame(num_attributes.apply( np.median )).T

# Dispersion - desvio padrão, mínimo, máximo, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( np.min ) ).T
d3 = pd.DataFrame( num_attributes.apply( np.max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max( ) - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew( ) ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T


# Concatenate

m1 = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m1.columns = ['attributes', 'min', 'max', 'range', 'mean', 'mediana', 'std', 'skew', 'kurtosis']
m1

,attributes,min,max,range,mean,mediana,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611121,2.08,96.759763,186.506973,59005.719804
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


### 1.7.2 Categorical Attributes

In [17]:
cat_attributes.head()

,invoice_no,stock_code,description,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
1,536365,71053,WHITE METAL LANTERN,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,United Kingdom


###  Invoice_No

In [18]:
# Problema: Temos invoice com letras e números

# Identificação >

df_letter_invoices = df1.loc[df1['invoice_no'].apply( lambda x : bool( re.search( '[^0-9]+', x ) ) ), :]

print('Total number of invoices: {}'.format( len( df_letter_invoices )))  
print('Total number of negative quantity: {}'.format( len(df_letter_invoices[ df_letter_invoices['quantity'] < 0])))

Total number of invoices: 9291
Total number of negative quantity: 9288


### Stock Code

In [19]:
# Check stock codes only characters
df1.loc[df1['stock_code'].apply( lambda x : bool( re.search( '^[a-zA-Z]+$', x ) ) ) ,'stock_code'].unique()

# Ação:
## 1. Remove stock_code in ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']


array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

### Description

In [20]:
df1.head()

#  Ação: Delete Description

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850


### Country

In [21]:
df1['country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [22]:
df1['country'].value_counts( normalize = True).head()

United Kingdom    0.914320
Germany           0.017521
France            0.015790
EIRE              0.015124
Spain             0.004674
Name: country, dtype: float64

In [23]:
df1[['customer_id','country']].drop_duplicates().groupby( 'country').count().reset_index().sort_values( 'customer_id', ascending = False).head()

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31


# 2.0. Filtragem de Variáveis

In [24]:
df2 = df1.copy()

In [25]:
df2.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int64
dtype: object

In [26]:
 # === Numerical attributes ====
df2 = df2.loc[df2['unit_price'] >= 0.04, :]

# === Categorical attributes ====
df2 = df2[~df2['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], )]

# description
df2 = df2.drop( columns='description', axis=1 )

# map 
df2 = df2[~df2['country'].isin( ['European Community', 'Unspecified' ] ) ]


# bad users
df2 = df2[~df2['customer_id'].isin( [16446] )]

# quantity
df2_returns = df2.loc[df1['quantity'] < 0, :]
df2_purchases = df2.loc[df1['quantity'] >= 0, :]





# 3.0. Feature Engineering

In [27]:
df3 = df2.copy()

## 3.1. Feature Creation

In [28]:
# Data Reference

df_ref = df3.drop( ['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country'], axis=1 ).drop_duplicates( ignore_index=True )

### 3.1.1 Gross Revenue

In [29]:
# Gross Revenue ( Faturamento ) quantity * price
df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']

# Monetary
df_monetary = df2_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby( 'customer_id' ).sum().reset_index()
df_ref = pd.merge( df_ref, df_monetary, on='customer_id', how='left' )

df_ref.isna().sum()

/home/leandro/.pyenv/versions/pa005insiderscluestering/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


customer_id       0
gross_revenue    91
dtype: int64

### 3.1.2 Recency - Day from last purchase

In [30]:
 # Recency - Last day purchase
df_recency = df2_purchases.loc[:, ['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = ( df2['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )

df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 3.1.4.1 Quantity of products purchased

In [31]:
 # Numero de produtos
df_freq = (df2_purchases.loc[:, ['customer_id', 'stock_code']].groupby( 'customer_id' ).count()
                                                           .reset_index()
                                                           .rename( columns={'stock_code': 'qtde_products'} ) )
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_products    91
dtype: int64

### 3.1.7 Number of  returns

In [32]:
# Number of Returns
df_returns = df2_returns [[ 'customer_id', 'quantity']].groupby( 'customer_id').sum().reset_index().rename( columns ={'quantity': 'qtde_returns'} )
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1

df_ref = pd.merge( df_ref, df_returns, how = 'left', on= 'customer_id')
df_ref.loc[ df_ref['qtde_returns'].isna(), 'qtde_returns'] = 0


df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_products    91
qtde_returns      0
dtype: int64

In [33]:
# Number of Returns
df2_returns [[ 'customer_id', 'quantity']].groupby( 'customer_id').sum().reset_index().rename( columns ={'quantity': 'qtde_returns'} )
                 

,customer_id,qtde_returns
0,12346,-74215
1,12352,-63
2,12359,-10
3,12362,-17
4,12375,-1
...,...,...
1590,22473,-1
1591,22485,-288
1592,22515,-324
1593,22610,-24


### 3.1.10 Frequency Purchase

In [34]:
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                            .groupby( 'customer_id')
                                                            .agg( max_ = ( 'invoice_date', 'max' ),
                                                                  min_ = ( 'invoice_date', 'min'),
                                                                  days_ = ('invoice_date', lambda x : ( ( x.max()- x.min() ).days) + 1 ) ,
                                                                  buy_ = ( 'invoice_no', 'count') ) ).reset_index()


# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply ( lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis = 1)

# Merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'frequency']], on = 'customer_id', how = 'left')

df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtde_products    91
qtde_returns      0
frequency        91
dtype: int64

In [35]:
df_ref.head()

,customer_id,gross_revenue,recency_days,qtde_products,qtde_returns,frequency
0,17850,5391.21,372.0,297.0,40.0,17.000000
1,13047,3232.59,56.0,171.0,35.0,0.028302
2,12583,6705.38,2.0,232.0,50.0,0.040323
3,13748,948.25,95.0,28.0,0.0,0.017921
4,15100,876.00,333.0,3.0,22.0,0.073171


# 4.0. Exploratory Data Analysis

In [36]:
df4 = df_ref.dropna()


## 4.3 Estudo do Espaço

In [37]:
# Selected dataset
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qtde_products', 'frequency', 'qtde_returns']
df43 = df4[cols_selected].drop( columns = 'customer_id', axis = 1)

In [38]:
df43.head()

,gross_revenue,recency_days,qtde_products,frequency,qtde_returns
0,5391.21,372.0,297.0,17.000000,40.0
1,3232.59,56.0,171.0,0.028302,35.0
2,6705.38,2.0,232.0,0.040323,50.0
3,948.25,95.0,28.0,0.017921,0.0
4,876.00,333.0,3.0,0.073171,22.0


In [39]:
mm = pp.MinMaxScaler()

#
df43[ 'gross_revenue' ]        = mm.fit_transform( df43 [['gross_revenue']])
df43[ 'recency_days' ]         = mm.fit_transform( df43[['recency_days']])
df43['qtde_products']          = mm.fit_transform( df43[['qtde_products']])
df43['qtde_returns']           = mm.fit_transform( df43[['qtde_returns']])
df43['frequency']              = mm.fit_transform( df43[['frequency']])


### 4.3.4 Tree-Based embedding

In [40]:
# Training dataset

X = df43.drop( columns = [ 'gross_revenue'], axis = 1 )
y = df43['gross_revenue']

# Model definittion
rf_model = en.RandomForestRegressor ( n_estimators = 100, random_state= 42)

# Model trainning
rf_model.fit( X,y)

#Leaf
df_leaf = pd.DataFrame( rf_model.apply( X ) )

# DataFrame Leaf



In [41]:
# Reduzer dimensionality
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( df_leaf )

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]


/home/leandro/.pyenv/versions/pa005insiderscluestering/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:260: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


# 5.0 Data Preparation

In [42]:
df5 = df_tree.copy( )

In [43]:
df5.to_csv(path+'src/data/tree_based_embedding.csv')

# 7.0. Hyperparameter Fine-tuning

In [44]:
X = df_tree.copy()

In [45]:
X.head()

,embedding_x,embedding_y
0,8.065233,-6.196950
1,10.827569,-2.550333
2,8.319387,-5.599104
3,-2.007851,12.433291
4,6.869921,26.271196


# 8.0. Model Training

## 8.1. Final Model

In [46]:
# Model Definition
k = 8
gmm_model = mx.GaussianMixture ( n_components = k, n_init = 300, random_state = 32)
# Model Training
gmm_model.fit(X)

# Clustering
labels  = gmm_model.predict( X )


## 8.2. Cluster Validation

In [47]:
## WSS ( Within-cluster sum of square)
#print( 'WSS value: {}'.format( kmeans.inertia_ ) )

## SS ( Silhouette Score )
print( 'SS value: {}'.format( m.silhouette_score( X, labels, metric='euclidean' ) ) )

SS value: 0.29330870509147644


# 9.0. Cluster Analysis

In [48]:
df92 = df4[cols_selected].copy()
df92['cluster'] = labels


# change dtypes
df92['recency_days'] = df92['recency_days'].astype( int )
df92['qtde_products'] = df92['qtde_products'].astype( int )
df92['qtde_returns'] = df92['qtde_returns'].astype( int )

from datetime import datetime

#df92['last_training_timestamp'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S')

In [49]:
# Number of customer
df_cluster = df92[['customer_id','cluster']].groupby( 'cluster' ).count().reset_index()
df_cluster['perc_customer'] = 100*(df_cluster['customer_id']/df_cluster['customer_id'].sum())

# Average gross revenue
df_avg_gross_revenue = df92[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_gross_revenue, how = 'inner', on = 'cluster')

# Average recency days
df_avg_recency_days = df92[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_recency_days, how = 'inner', on = 'cluster')

# Quantidade de produtos
df_qtde_products = df92[['qtde_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qtde_products, how = 'inner', on = 'cluster')

# Frequency
df_frequency = df92[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_frequency, how = 'inner', on = 'cluster')


# returns
df_qtde_returns = df92[['qtde_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qtde_returns, how = 'inner', on = 'cluster')


df_cluster.sort_values( 'gross_revenue', ascending = False)

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qtde_products,frequency,qtde_returns
2,2,956,16.786655,5907.257552,60.316946,317.376569,0.282462,161.792887
6,6,1380,24.231782,1807.489312,114.607246,96.245652,0.574754,4.694928
1,1,282,4.951712,1170.371064,61.496454,56.124113,0.027563,10.106383
3,3,324,5.689201,804.382994,88.027778,30.320988,0.237213,4.466049
0,0,271,4.758560,647.048967,50.110701,14.516605,0.024271,1.025830
5,5,795,13.959614,638.717195,116.748428,35.982390,0.607577,6.622642
7,7,880,15.452151,482.162602,166.701136,22.654545,0.842358,4.735227
4,4,807,14.170325,328.237447,187.270136,16.175960,0.914689,3.609665


02 Cluster Insiders

06 Cluster More Products

01 Cluster Spend Money

03 Cluster Even More Products

00 Cluster Less Days

05 Cluster 1K

07 Cluster Stop Returnres

04 Cluster More Buy

### Cluster 01:  ( Candidato a Insider )
 
- Número de customers: 468 (16% do customers )
- Faturamento médio: 8836
- Recência média: 21 dias
- Média de Produtos comprados: 424 produtos
- Frequência de Produtos comprados: 0.09 produtos/dia
- Receita em média: $8836.13,00 dólares
 
### Cluster 02:  
 
 - Número de customer: 31 (0.7% dos customers)
 - Recência em média: 14 dias
 - Compras em média: 53 compras
 - Receita em média: $ 40.543,00.
 
 ### Cluster 03: 
 
 - Número de customer: 4.335 (99% dos customers)
 - Recência em média: 92 dias
 - Compras em média: 05 compras
 - Receita em média: $ 1.372,57.

# 11.0. Deploy to Production

In [50]:
import sqlite3
from sqlalchemy import create_engine


In [51]:
df92.head()

,customer_id,gross_revenue,recency_days,qtde_products,frequency,qtde_returns,cluster
0,17850,5391.21,372,297,17.000000,40,2
1,13047,3232.59,56,171,0.028302,35,2
2,12583,6705.38,2,232,0.040323,50,2
3,13748,948.25,95,28,0.017921,0,5
4,15100,876.00,333,3,0.073171,22,7


In [52]:
# # create table
# query_create_table_insiders = """
#    CREATE TABLE insiders ( 
#        customer_id     INTEGER,
#        gross_revenue   REAL,
#        recency_days    INTEGER,    
#        qtde_products   INTEGER,
#        frequency       REAL,
#        qtde_returns    INTEGER,
#        cluster         INTEGER
#    )
# """


# conn = sqlite3.connect( 'insiders_db.sqlite' )
# conn.execute( query_create_table_insiders )
# conn.commit()
# conn.close()

# Insert Data
conn = create_engine( 'sqlite:///insiders_db.sqlite')
df92.to_sql( 'insiders', con = conn, if_exists= 'append', index = False)


# Select Data

In [53]:
# # get query
# query = """
#         SELECT * FROM insiders
# """

# df = pd.read_sql_query( query, conn)
# df.head()